In [1]:
import requests
import re
import json
from lxml import etree
import time
import os

# url="https://weibo.com/2447680824/G5nMd0MBJ?type=comment#_rnd1519906057635"
#模拟游客登录获取cookies
class visitor():
    def __init__(self,url):
        try:
            self.cookies,self.headers=self.get_cookies()
            self.id=self.weibo_spider(url)
            self.fail=False
        except:
            print("模拟失败-。-")
            self.fail=True


    def get_cookies(self):
        # 获取dict_data
        print("正在模拟游客登录")
        S = requests.session()
        url = "https://passport.weibo.com/visitor/genvisitor"
        S.headers.update(
            {
                "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:41.0) Gecko/20100101 Firefox/41.0"
            }
        )

        data = {
            "cb": "gen_callback",
            "fp": '{"os":"1","browser":"Gecko60,0,0,0","fonts":"undefined","screenInfo":"1536*864*24","plugins":""}'
        }
        response = S.post(url, data=data)
        pattren=re.compile(r"\((.*)\)")
        data=pattren.findall(response.text)[0]
        
        #获取json中data
        dict_data=json.loads(data)["data"]
        tid=dict_data["tid"]
        confidence=dict_data["confidence"]
        where=dict_data["new_tid"]
        if where:
            where=3
        else:
            where=2
        while(len(str(confidence))<3):
            confidence="0"+str(confidence)
        #tid="KCEsfUfkAmyXExt9tiPN61f32Vvh4wViWQaeHptBZLc="
        #手动编码格式转换
        tid=tid.replace("+","%2b")
        tid=tid.replace("=","%3d")

        url="https://passport.weibo.com/visitor/visitor?a=incarnate"\
            "&t="+str(tid)+ \
            "&w=" + str(where) + \
            "&c="+str(confidence)+\
            "&gc="\
            "&cb=cross_domain" \
            "&from=weibo"
        response=S.get(url)
        data=pattren.findall(response.text)[0]
        #https://passport.weibo.com/visitor/visitor?a=incarnate&t=hVEmh0nd32++OFXP3wiB6b05C9A5L38fmq7ArFKTNq8=&w=2&c=095&gc=&cb=cross_domain&from=weibo'
        #https://passport.weibo.com/visitor/visitor?a=incarnate&t=+A1gVsii+zY9OI9v/e+o1lfhlTPQ20U3Fkuz8nn/7rU=&w=2&c=095&gc=&cb=cross_domain&from=weibo&_rand=0.42337865580692513
        dict_data = json.loads(data)
        if "succ" not in dict_data["msg"]:
            printf("tid不合法")
            printf("dict_data:",dict_data)
            self.fail=True
            return None,None
        dict_data=dict_data["data"]
        sub=dict_data["sub"]    #没有
        subp=dict_data["subp"]
        url="https://login.sina.com.cn/visitor/visitor?a=crossdomain&cb=return_back"\
            +"&s="+str(sub)\
            +"&sp="+str(subp)\
            +"&from=weibo"
        response=S.get(url)
        print("成功获取游客Cookies")
        return S.cookies,S.headers

    #base62解码
    def base62(self,string):
        alphabet = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        base = len(alphabet)
        strlen = len(string)
        num = 0
        idx = 0
        for char in string:
            power = (strlen - (idx + 1))
            num += alphabet.index(char) * (base ** power)
            idx += 1
        return num

    def weibo_spider(self,url):
        S=requests.session()
        S.cookies=self.cookies
        S.headers=self.headers
        response=S.get(url)
        #获取微博大致内容
        selector = etree.HTML(response.text)
        weibo=selector.xpath('/title/text()')
        page = etree.HTML(response.text)
        content = page.xpath('//title/text()')[0]
        content=str(content).replace("\n"," ")
        print("爬取ing\n\t",content)
        
        # #获取异步加载url中的id
        pattern=re.compile(r"\d\/(.*)\?.*type")
        content=pattern.search(url).group(1)
        string1=str(self.base62(content[0]))
        string2=str(self.base62(content[1:5]))
        while(len(string2)<7):
            string2="0"+string2
        string3=str(self.base62(content[5:]))
        while (len(string3) < 7):
            string3 = "0" + string3
        id=string1+string2+string3
        return id

    def catch_comments(self,page=1,past=None):
        
        #模拟异步加载
        #原评论url：https://weibo.com/1537790411/Fpp6ImeOh?filter=hot&root_comment_id=0&type=comment
        path="./data"
        if(page==1):
            print("开始爬取~")
            #判断是否存在该文件夹
            if not os.path.exists(path):
                os.mkdir(path)
        S=requests.session()
        S.cookies=self.cookies
        S.headers=self.headers
        
        #异步加载url：https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4160547165300149&from=singleWeiBo&__rnd=1547636454812
        url="https://weibo.com/aj/v6/comment/big?ajwvr=6"\
            +"&id="+str(self.id)\
            +"&page="+str(page) \
            + "&filter=hot" \
            +"&from=singleWeiBo"
        #输出该新闻评论所在的网页位置
        #print(url)
        
        response=S.get(url)
        html=json.loads(response.text)["data"]["html"]
        #如果两次相同表示结束了 -。-
        # if past_html==html:
        #     print("爬取结束")
        #     # print(self.id)
        #     print("共",page,"页")
        #     return
        #搜索评论
        text=etree.HTML(html)

        #评论数-xpath
        # comments=text.xpath('//div[@class="list_li S_line1 clearfix"]//div[@class="WB_text"]//text()')
        comments = text.xpath('//div[@class="list_li S_line1 clearfix"]/*/div[@class="WB_text"]')
        # 评论数
        points = text.xpath('//div[@class="list_li S_line1 clearfix"]//*/span[@node-type="like_status"]/child::*[2]//text()')
        #点赞数-xpath
        # points
        pattern = re.compile(r'\：(.*)')
        # try:
        if page==1:
            wa="w"
        else:
            wa="a"
        f=open("./data/sina.txt", wa,encoding='utf-8')
        for i in range(len(comments)):
            comment = comments[i].xpath("text()")
            comment = ",".join(comment[1:])[1:].strip()
            point = points[i]
            if i==0:
                now = {"len": len(comments), "comment":comment}
                if now==past:
                    print("爬取结束")
                    return
                else:
                    past=now
            if point == "赞":
                point = "0"
            #点赞数为权重0.2
            weights=int(0.2*int(point))
            #写入评论
            comment=comment+"\n"
            f.write(comment)
            for i in range(weights):
                f.write(comment)
            #写入点赞数
        print("已写入", page, "页")
        f.close()
    # except :
    #     print("写入文件失败")

        page+=1
        self.catch_comments(page,past)

    
#模块调用
def start(url=None):
    if url==None:
        print("请输入正确的url")
        return
    else:
        spider=visitor(url)
        spider.catch_comments()

if __name__=="__main__":
    url = input("输入需要爬取的微博url:\n")
    spider=visitor(url)
    if spider.fail==False:
        spider.catch_comments()
#https://weibo.com/1537790411/Fpp6ImeOh?filter=hot&root_comment_id=0&type=comment


输入需要爬取的微博url:
https://weibo.com/1537790411/Fpp6ImeOh?filter=hot&root_comment_id=0&type=comment
正在模拟游客登录
成功获取游客Cookies
爬取ing
	 大家好，给大家介绍一下，这是我女朋友@关... 来自M鹿M - 微博
开始爬取~
已写入 1 页
已写入 2 页
已写入 3 页
已写入 4 页
已写入 5 页
已写入 6 页
已写入 7 页
已写入 8 页
已写入 9 页
已写入 10 页
已写入 11 页
已写入 12 页
已写入 13 页
已写入 14 页
已写入 15 页
已写入 16 页
已写入 17 页
已写入 18 页
已写入 19 页
已写入 20 页
已写入 21 页
已写入 22 页
已写入 23 页
已写入 24 页
已写入 25 页
已写入 26 页
已写入 27 页
已写入 28 页
已写入 29 页
已写入 30 页
已写入 31 页
已写入 32 页
已写入 33 页
已写入 34 页
已写入 35 页
已写入 36 页
已写入 37 页
已写入 38 页
已写入 39 页
已写入 40 页
已写入 41 页
已写入 42 页
已写入 43 页
已写入 44 页
已写入 45 页
已写入 46 页
已写入 47 页
已写入 48 页
已写入 49 页
已写入 50 页
已写入 51 页
已写入 52 页
已写入 53 页
已写入 54 页
已写入 55 页
已写入 56 页
已写入 57 页
已写入 58 页
已写入 59 页
已写入 60 页
已写入 61 页
已写入 62 页
已写入 63 页
已写入 64 页
已写入 65 页
已写入 66 页
已写入 67 页
已写入 68 页
已写入 69 页
已写入 70 页
已写入 71 页
已写入 72 页
已写入 73 页
已写入 74 页
已写入 75 页
已写入 76 页
已写入 77 页
已写入 78 页
已写入 79 页
已写入 80 页
已写入 81 页
已写入 82 页
已写入 83 页
已写入 84 页
已写入 85 页
已写入 86 页
已写入 87 页
已写入 88 页
已写入 89 页
已写入 90 页
已写入 91 页
已写入 92 页
已写入 93 页
已写入 9

KeyboardInterrupt: 